# Import package

In [5]:
pip list

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
absl-py (0.11.0)
Adafruit-GPIO (1.0.4)
Adafruit-MotorHAT (1.4.0)
Adafruit-PureIO (1.1.7)
Adafruit-SSD1306 (1.6.2)
appdirs (1.4.4)
argon2-cffi (20.1.0)
astor (0.8.1)
async-generator (1.10)
attrs (20.3.0)
backcall (0.2.0)
bleach (3.2.1)
certifi (2020.11.8)
cffi (1.14.3)
chardet (3.0.4)
Cython (0.29.21)
dataclasses (0.7)
decorator (4.4.2)
defusedxml (0.6.0)
entrypoints (0.3)
funcsigs (1.0.2)
future (0.17.1)
futures (3.1.1)
gast (0.2.2)
google-pasta (0.2.0)
grpcio (1.33.2)
h5py (2.10.0)
idna (2.10)
importlib-metadata (2.0.0)
ipykernel (5.3.4)
ipython (7.16.1)
ipython-genutils (0.2.0)
ipywidgets (7.5.1)
jedi (0.17.2)
jetbot (0.4.1)
Jetson.GPIO (2.0.15)
Jinja2 (2.11.2)
json5 (0.9.5)
jsonschema (3.2.0)
jupyter (1.0.0)
jupyter-clickable-image-widget (0.1.0a0, /root/jupyter_clickable_i

In [4]:
pip install Jetson.GPIO

  Running setup.py bdist_wheel for Jetson.GPIO ... done
  Stored in directory: /root/.cache/pip/wheels/b2/43/d4/efa32294f0705bdd848ce2b655cf8409e30004435bd6c37f8c
Successfully built Jetson.GPIO
Note: you may need to restart the kernel to use updated packages.


In [6]:
import time
import Jetson.GPIO as GPIO
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BOARD)

# Define

## GPIO pin maps

In [22]:
# Enable Pin 32 / PWM0 from terminal 
# sudo busybox devmem 0x700031fc 32 0x45
# sudo busybox devmem 0x6000d504 32 0x2

# Enable Pin 33 / PWM2
# sudo busybox devmem 0x70003248 32 0x46
# sudo busybox devmem 0x6000d100 32 0x00

/bin/bash: sudo: command not found
/bin/bash: sudo: command not found


In [27]:
left_motor_pwm_pin = 32
left_motor_ctrl_pins = [35,36]
right_motor_pwm_pin = 33
right_motor_ctrl_pins = [37,38]

GPIO.setup(left_motor_pwm_pin, GPIO.OUT)
left_motor_pwm = GPIO.PWM(left_motor_pwm_pin, 50) # (channel, frequency)
left_motor_pwm.start(0)
for pin in left_motor_ctrl_pins:
    GPIO.setup(pin,GPIO.OUT,initial=GPIO.LOW)
GPIO.setup(right_motor_pwm_pin, GPIO.OUT)
right_motor_pwm = GPIO.PWM(right_motor_pwm_pin, 50)
# right_motor_pwm.start(0)
for pin in right_motor_ctrl_pins:
    GPIO.setup(pin,GPIO.OUT,initial=GPIO.LOW)

In [17]:
for n in range(10):
    GPIO.output(left_motor_ctrl[0], not GPIO.input(left_motor_ctrl[0]))
    time.sleep(0.5)

In [31]:
left_motor_pwm.ChangeDutyCycle(0.2)

In [36]:
GPIO.output(right_motor_ctrl[0], GPIO.LOW)
GPIO.output(right_motor_ctrl[1], GPIO.HIGH)

In [33]:
GPIO.output(right_motor_ctrl[0], GPIO.HIGH)
GPIO.output(right_motor_ctrl[1], GPIO.LOW)

In [37]:
GPIO.output(right_motor_ctrl[0], GPIO.LOW)
GPIO.output(right_motor_ctrl[1], GPIO.LOW)

In [35]:
GPIO.output(right_motor_ctrl[0], GPIO.HIGH)
GPIO.output(right_motor_ctrl[1], GPIO.HIGH)

In [1]:
import time
import Jetson.GPIO as GPIO
import traitlets
from traitlets.config.configurable import Configurable

GPIO.setwarnings(False)
GPIO.setmode(GPIO.BOARD)

class DCMotor(Configurable):
    value = traitlets.Float()    
    
    def __init__(self, pwm_pin, ctrl_pin1, ctrl_pin2, **kwargs):
        self.pwm_pin = pwm_pin
        self.ctrl_pin1 = ctrl_pin1
        self.ctrl_pin2 = ctrl_pin2
        # Motor calibration
        self.alpha = kwargs.get('alpha', 1.0)
        self.beta = kwargs.get('beta', 0.0)
        # GPIO setup
        GPIO.setup(self.pwm_pin, GPIO.OUT)
        self.pwm = GPIO.PWM(self.pwm_pin, 50) # (channel, frequency)
        self.pwm.start(0)        
        self.speed = 0
        GPIO.setup(self.ctrl_pin1,GPIO.OUT,initial=GPIO.LOW)
        GPIO.setup(self.ctrl_pin2,GPIO.OUT,initial=GPIO.LOW)
    
    @traitlets.observe('value')
    def _observe_value(self, change):
        self._write_value(change['new'])

    def _write_value(self, value):
        """Sets motor value between [-1, 1]"""
        mapped_value = int(100.0 * (self.alpha * value + self.beta))
        speed = min(max(abs(mapped_value), 0), 100)
        self.set_speed(speed)
        if mapped_value < 0:
            self.rotate_forward()
        else:
            self.rotate_backward()
            
    def set_speed(self, normalized_speed):
        # normalized_speed in percentange from 0 - 100
        self.pwm.ChangeDutyCycle(normalized_speed)
        self.speed = normalized_speed
        
    def rotate_forward(self):
        GPIO.output(self.ctrl_pin1, GPIO.LOW)
        GPIO.output(self.ctrl_pin2, GPIO.HIGH)
    
    def rotate_backward(self):
        GPIO.output(self.ctrl_pin1, GPIO.HIGH)
        GPIO.output(self.ctrl_pin2, GPIO.LOW)
    
    def stop(self):
        GPIO.output(self.ctrl_pin1, GPIO.LOW)
        GPIO.output(self.ctrl_pin2, GPIO.LOW)

right_motor = DCMotor(33,35,37, alpha = 0.8)
left_motor = DCMotor(32,38,36)


In [2]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Controller()

In [3]:
left_link = traitlets.dlink((controller.axes[1], 'value'), (left_motor, 'value'))
right_link = traitlets.dlink((controller.axes[3], 'value'), (right_motor, 'value'))

In [2]:
right_motor.set_speed(100)
left_motor.set_speed(100)
right_motor.rotate_forward()
left_motor.rotate_forward()

In [22]:
right_motor.set_speed(80)
left_motor.set_speed(80)
right_motor.rotate_backward()
left_motor.rotate_backward()

In [3]:
left_motor.stop()
right_motor.stop()

In [18]:
right_motor.set_speed(60)
# right_motor.rotate_forward()


In [4]:
import time
import Jetson.GPIO as GPIO
import traitlets
from traitlets.config.configurable import Configurable

GPIO.setwarnings(False)
GPIO.setmode(GPIO.BOARD)

class Motor(Configurable):

    value = traitlets.Float()
    
    # config
    alpha = traitlets.Float(default_value=1.0).tag(config=True)
    beta = traitlets.Float(default_value=0.0).tag(config=True)

    def __init__(self, driver, channel, *args, **kwargs):
        super(Motor, self).__init__(*args, **kwargs)  # initializes traitlets

        self._driver = driver
        self._motor = self._driver.getMotor(channel)
        if(channel == 1):
            self._ina = 1
            self._inb = 0
        else:
            self._ina = 2
            self._inb = 3
        atexit.register(self._release)
        
    @traitlets.observe('value')
    def _observe_value(self, change):
        self._write_value(change['new'])

    def _write_value(self, value):
        """Sets motor value between [-1, 1]"""
        mapped_value = int(255.0 * (self.alpha * value + self.beta))
        speed = min(max(abs(mapped_value), 0), 255)
        self._motor.setSpeed(speed)
        if mapped_value < 0:
            self._motor.run(Adafruit_MotorHAT.FORWARD)
            # The two lines below are required for the Waveshare JetBot Board only
            self._driver._pwm.setPWM(self._ina,0,0)
            self._driver._pwm.setPWM(self._inb,0,speed*16)
        else:
            self._motor.run(Adafruit_MotorHAT.BACKWARD)
            # The two lines below are required for the Waveshare JetBot Board only
            self._driver._pwm.setPWM(self._ina,0,speed*16)
            self._driver._pwm.setPWM(self._inb,0,0)

    def _release(self):
        """Stops motor by releasing control"""
        self._motor.run(Adafruit_MotorHAT.RELEASE)
        # The two lines below are required for the Waveshare JetBot Board only
        self._driver._pwm.setPWM(self._ina,0,0)
        self._driver._pwm.setPWM(self._inb,0,0)